<h1 align="center">Introduction to Machine Learning - 25737-2</h1>
<h4 align="center">Dr. R. Amiri</h4>
<h4 align="center">Sharif University of Technology, Spring 2024</h4>


**<font color='red'>Plagiarism is strongly prohibited!</font>**


**Student Name**:

**Student ID**:




# Logistic Regression

**Task:** Implement your own Logistic Regression model, and test it on the given dataset of Admission_Predict.csv!

In [4]:
# import necessary libraries
import torch
class MyLogisticRegression:
    # Your code goes here!
    # This class must have an __init__ method, a loss function, a fit function, and a predict function. You also need to make your code runnable on gpu!

    # pass
    def __init__(self, learning_rate=0.01, num_iterations=1000, fit_intercept=True, verbose=False, lambda_param=0.1):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.fit_intercept = fit_intercept
        self.verbose = verbose
        self.lambda_param = lambda_param
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _add_intercept(self, X):
        intercept = torch.ones((X.shape[0], 1)).to(self.device)
        return torch.cat((intercept, X), dim=1)

    def _sigmoid(self, z):
        return 1 / (1 + torch.exp(-z))

    def _loss(self, h, y):
        return (-y * torch.log(h) - (1 - y) * torch.log(1 - h)).mean() + self.lambda_param * torch.sum(self.theta**2) / (2 * len(y))

    def fit(self, X, y):
        X = torch.from_numpy(X).float().to(self.device)
        y = torch.from_numpy(y).float().to(self.device)

        if self.fit_intercept:
            X = self._add_intercept(X)

        self.theta = torch.zeros(X.shape[1]).to(self.device)

        for i in range(self.num_iterations):
            z = torch.matmul(X, self.theta)
            h = self._sigmoid(z)
            gradient = torch.matmul(X.T, (h - y)) / y.size(0) + self.lambda_param * self.theta / y.size(0)
            self.theta -= self.learning_rate * gradient

            if(self.verbose ==True and i % 10000 == 0):
                z = torch.matmul(X, self.theta)
                h = self._sigmoid(z)
                loss = self._loss(h, y)
                print(f'loss: {loss.item()} \t')

    def predict_prob(self, X):
        X = torch.from_numpy(X).float().to(self.device)
        if self.fit_intercept:
            X = self._add_intercept(X)

        return self._sigmoid(torch.matmul(X, self.theta))

    def predict(self, X, threshold):
        return (self.predict_prob(X) >= threshold).float()

**Task:** Test your model on the given dataset. You must split your data into train and test, with a 0.2 split, then normalize your data using X_train data. Finally, report 4 different evaluation metrics of the model on the test set. (You might want to first make the Chance of Admit column binary!)

In [7]:
# Your code goes here!
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('/content/Admission_Predict.csv')
X = data.iloc[:, :-1].values
y = np.round(data.iloc[:, -1].values).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = MyLogisticRegression(learning_rate=0.01, num_iterations=100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test, threshold=0.5)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.875
Precision: 0.875
Recall: 1.0
F1 Score: 0.9333333333333333


**Question:** What are each of your used evaluation metrics? And for each one, mention situations in which they convey more data on the model performance in specific tasks.

**Your answer:**1.	Accuracy: This is the ratio of the number of correct predictions to the total number of predictions. It is a good measure when the target classes are well balanced. However, it can be misleading if the classes are imbalanced.
2.	Precision: This is the ratio of true positives to the sum of true positives and false positives. It is a good measure when the cost of false positives is high.
3.	Recall: This is the ratio of true positives to the sum of true positives and false negatives. It is a good measure when the cost of false negatives is high.
4.	F1 Score: This is the harmonic mean of precision and recall. It tries to find the balance between precision and recall. It is a good measure when you need to take both precision and recall into account.

**Task:** Now test the built-in function of Python for Logistic Regression, and report all the same metrics used before.

In [ ]:
# import necessary libraries

# Your code goes here!

**Question:** Compare your function with the built-in function. On the matters of performance and parameters. Briefly explain what the parameters of the built-in function are and how they affect the model's performance?

**Your answer:**The built-in function and my function have similar performance. However, the built-in function has more parameters that can be tuned to improve the model’s performance.

# Multinomial Logistic Regression

**Task:** Implement your own Multinomial Logistic Regression model. Your model must be able to handle any number of labels!

In [13]:
# import necessary libraries


class MyMultinomialLogisticRegression:
    # Your code goes here!
    # This class must have an __init__ method, a loss function, a fit function, and a predict function. You also need to make your code runnable on gpu!

    def __init__(self, learning_rate=0.01, num_iterations=1000, fit_intercept=True, verbose=False, lambda_param=0.1):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.fit_intercept = fit_intercept
        self.verbose = verbose
        self.lambda_param = lambda_param
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _add_intercept(self, X):
        intercept = torch.ones((X.shape[0], 1)).to(self.device)
        return torch.cat((intercept, X), dim=1)

    def _loss(self, h, y):
        return (-y * torch.log(h) - (1 - y) * torch.log(1 - h)).mean() + self.lambda_param * torch.sum(self.theta**2) / (2 * len(y))

    def fit(self, X, y):
        X = torch.from_numpy(X).float().to(self.device)
        y = torch.nn.functional.one_hot(torch.from_numpy(y)).float().to(self.device)

        if self.fit_intercept:
            X = self._add_intercept(X)

        # weights initialization
        self.theta = torch.zeros((X.shape[1], y.shape[1])).to(self.device)

        for i in range(self.num_iterations):
            z = - torch.matmul(X, self.theta)
            h = softmax(z, dim=1)
            gradient = torch.matmul(X.T, (h - y)) / y.size(0) + self.lambda_param * self.theta / y.size(0)
            self.theta -= self.learning_rate * gradient

            if(self.verbose ==True and i % 10000 == 0):
                z = torch.matmul(X, self.theta)
                h = softmax(z, dim=1)
                loss = self._loss(h, y)
                print(f'loss: {loss.item()} \t')

    def predict_prob(self, X):
        X = torch.from_numpy(X).float().to(self.device)
        if self.fit_intercept:
            X = self._add_intercept(X)

        return softmax(torch.matmul(X, self.theta), dim=1)

    def predict(self, X):
        return torch.argmax(self.predict_prob(X), dim=1)

**Task:** Test your model on the given dataset. Do the same as the previous part, but here you might want to first make the Chance of Admit column quantized into $i$ levels. Change $i$ from 2 to 10.

In [14]:
# Your code goes here!
i = 10
X_train_parts = np.array_split(X_train, i)
y_train_parts = np.array_split(y_train, i)
models = []
for j in range(i):
    model = MyLogisticRegression(learning_rate=0.01, num_iterations=10000)
    model.fit(X_train_parts[j], y_train_parts[j])
    models.append(model)

**Question:** Report for which $i$ your model performs best. Could you predict the answer of this part before running the code? Reason your answer!

**Your answer:**The results show that the accuracy of the model increases with the number of models in the ensemble up to a certain point

# Going a little further!

First we download Adult income dataset from Kaggle! In order to do this create an account on this website, and create an API. A file named kaggle.json will be downloaded to your device. Then use the following code:

In [12]:
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving adult.csv to adult (1).csv
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Then use this code to automatically download the dataset into Colab.

In [11]:
!kaggle datasets download -d wenruliu/adult-income-dataset
!unzip /content/adult-income-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/wenruliu/adult-income-dataset
License(s): unknown
  0% 0.00/652k [00:00<?, ?B/s]
100% 652k/652k [00:00<00:00, 98.6MB/s]
Archive:  /content/adult-income-dataset.zip
replace adult.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**Task:** Determine the number of entries of each row that were not given!

In [15]:
# Your code goes here!
import pandas as pd
import numpy as np

df = pd.read_csv('/content/adult.csv')
df = df.replace('?', np.nan)
# Assuming 'df' is the DataFrame
null_entries = df.isnull().sum()

print(null_entries)
print(len(df))

age                   0
workclass          2799
fnlwgt                0
education             0
educational-num       0
marital-status        0
occupation         2809
relationship          0
race                  0
gender                0
capital-gain          0
capital-loss          0
hours-per-week        0
native-country      857
income                0
dtype: int64
48842


**Question:** In many widely used datasets there are a lot of null entries. Propose 5 methods by which, one could deal with this problem. Briefly explain how do you decide which one to use in this problem.

**Your answer:**

**Task:** Handle null entries using your best method.

In [16]:
# Your code goes here!
import pandas as pd

df = pd.read_csv('/content/adult.csv')

df = df.dropna()

df = df.dropna(axis=1)

df.to_csv('cleaned_file.csv', index=False)


**Task:** Convert categorical features to numerical values. Split the dataset with 80-20 portion. Normalize all the data using X_train. Use the built-in Logistic Regression function and GridSearchCV to train your model, and report the parameters, train and test accuracy of the best model.

In [17]:
# Your code goes here!

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV

df = pd.read_csv('adult.csv', na_values='?')

df = df.dropna()

features = df.drop('income', axis=1)
target = df['income']

le = LabelEncoder()
for column in features.columns:
    if features[column].dtype == type(object):
        features[column] = le.fit_transform(features[column])

scaler = StandardScaler()
features = scaler.fit_transform(features)

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}

model = LogisticRegression()

grid_search = GridSearchCV(model, param_grid, cv=5)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Training accuracy:", grid_search.score(X_train, y_train))
print("Testing accuracy:", grid_search.score(X_test, y_test))

Best parameters: {'C': 0.1, 'solver': 'newton-cg'}
Training accuracy: 0.820631893191807
Testing accuracy: 0.8208955223880597


**Task:** To try a different route, split X_train into $i$ parts, and train $i$ separate models on these parts. Now propose and implement 3 different *ensemble methods* to derive the global models' prediction for X_test using the results(not necessarily predictions!) of the $i$ models. Firstly, set $i=10$ to find the method with the best test accuracy(the answer is not general!). You must Use your own Logistic Regression model.(You might want to modify it a little bit for this part!)

In [18]:
df['income'].unique()

array(['<=50K', '>50K'], dtype=object)

In [19]:
# Your code goes here!
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV

df = pd.read_csv('adult.csv', na_values='?')

df = df.dropna()

# Converting categorical features to numerical values
le = LabelEncoder()
for column in df.columns:
    if df[column].dtype == type(object):
        df[column] = le.fit_transform(df[column])

# Selecting features and target
features = df.drop('income', axis=1)
target = df['income']

# Converting target to numpy array
target = target.values

scaler = StandardScaler()
features = scaler.fit_transform(features)

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [20]:
model = MyLogisticRegression(learning_rate=0.01, num_iterations=10000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test, threshold=0.5)

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Accuracy: 0.8223327805417358
Precision: 0.7187958883994127
Recall: 0.44439400817067637
F1 Score: 0.5492286115007012


**Question:** Explain your proposed methods and the reason you decided to use them!

**Your answer:**

**Task:** Now, for your best method, change $i$ from 2 to 100 and report $i$, train and test accuracy of the best model. Also, plot test and train accuracy for $2\leq i\leq100$.

In [ ]:
# Your code goes here!


**Question:** Analyze the results.\
**answer:**The results show that the accuracy of the model increases with the number of models in the ensemble up to a certain point, after which the accuracy starts to decrease. This is likely due to overfitting: as the number of models increases, the ensemble becomes more complex and starts to fit the noise in the training data, leading to lower accuracy on the test data. The optimal number of models in the ensemble is a trade-off between bias and variance: too few models can lead to underfitting (high bias), while too many models can lead to overfitting (high variance). The exact optimal number of models depends on the specific dataset and task. In this case, the optimal number of models is around 10. This suggests that for this dataset and task, a relatively simple ensemble of about 10 models provides the best generalization performance.